In [2]:
from snowflake.snowpark.session import Session
import snowflake.snowpark.types as T
import toml
import pandas as pd
import json

In [3]:
import toml  # Import the toml library

# Load secrets from the TOML file
secrets = toml.load("secrets.toml")

snowflake_secrets = secrets["SNOWFLAKE"]

accountname = snowflake_secrets["account"]
user = snowflake_secrets["user"]
password = snowflake_secrets["password"]
role = snowflake_secrets["role"]
database = snowflake_secrets["database"]
schema = snowflake_secrets["schema"]
warehouse = snowflake_secrets["warehouse"]

connection_parameters = {
    "account": accountname,
    "user": user,
    "password": password,
    "role": role,
    "database": database,
    "schema": schema,
    "warehouse": warehouse,
    "ocsp_fail_open": "False"
}

session = Session.builder.configs(connection_parameters).create()


In [3]:
print(session.sql('select current_warehouse(), current_database(), current_schema()').collect())

[Row(CURRENT_WAREHOUSE()='PRICE_PRED', CURRENT_DATABASE()='PRICE_PRED_DB', CURRENT_SCHEMA()='PRICE_PRED_SCHEMA')]


In [4]:
session.sql('CREATE OR REPLACE STAGE ML_MODELS').collect()

[Row(status='Stage area ML_MODELS successfully created.')]

In [5]:
pd.DataFrame(session.sql('SHOW STAGES').collect())

,created_on,name,database_name,schema_name,url,has_credentials,has_encryption_key,owner,comment,region,type,cloud,notification_channel,storage_integration,endpoint,owner_role_type,directory_enabled
0,2023-12-12 06:54:31.449000-08:00,ML_MODELS,PRICE_PRED_DB,PRICE_PRED_SCHEMA,,N,N,ACCOUNTADMIN,,None,INTERNAL,None,None,None,None,ROLE,N
1,2023-12-10 23:27:13.693000-08:00,PRICE_PRED_AWS_STAGE,PRICE_PRED_DB,PRICE_PRED_SCHEMA,s3://web-scrapping-bucket/,N,N,ACCOUNTADMIN,,us-east-2,EXTERNAL,AWS,None,AWS_S3_INTEGRATION,None,ROLE,N


In [29]:
def sproc_predict_using_regression(session: Session,
                                   training_table: str,
                                   include_history: str,
                                   period: int) -> T.Variant:
    # Loading data into pandas dataframe
    data_sdf = session.table(training_table)
    data = data_sdf.select('DATE', 'CLOSE','OPEN').to_pandas()
    data.drop_duplicates(subset='DATE', keep="last", inplace=True)
    data.dropna(subset=['DATE'])

    data.sort_values(by='DATE', inplace=True)
    data.columns = ['ds', 'y','X']
    #print(data)

    
    from sklearn.linear_model import LinearRegression
    # Actual model training
    
    model = LinearRegression()
    
    X = data['X'].values.reshape(-1, 1)  # Use this if you created 'ds_timestamp'
    
    y = data['y']
    
    model.fit(X, y)
    
    
    
    
    if include_history == 'Y':
        flag = True
    else:
        flag = False
 
    
    forecast = model.predict(X)
    
    result = { 'forecast': forecast.tolist()}
    
    return result
    


session.add_packages('snowflake-snowpark-python','scikit-learn')
# Registering the function as a Stored Procedure
sproc_predict_using_regression = session.sproc.register(func=sproc_predict_using_regression,
                                                       name='sproc_predict_using_regression',
                                                       is_permanent=True,
                                                       replace=True,
                                                       stage_location='@ML_MODELS',
                                                       packages=['snowflake-snowpark-python','scikit-learn'])
 
training_table = 'historical_prices'
show_history = 'N'
future_days = 4
 
pred_result = session.sql(
    "call sproc_predict_using_regression('{}', '{}', {})".format(training_table, show_history, future_days)
).collect()
'''
# Accessing the prediction results
future_dates = pred_result[0][0]
forecast_values = pred_result[0][0]
 
#pred_df = pd.DataFrame({'DATE': future_dates, 'PRICE': forecast_values})
#pred_df['DATE'] = pred_df['DATE'].dt.date
print(future_dates,forecast_values)
# Access pred_df for the predictions
'''
pred_df = pd.DataFrame(json.loads(pred_result[0][0]))
data_sdf = session.table(training_table)
data = data_sdf.select('DATE', 'CLOSE','OPEN').to_pandas()
data.drop_duplicates(subset='DATE', keep="last", inplace=True)
data.dropna(subset=['DATE'])

data.sort_values(by='DATE', inplace=True)
data.columns = ['ds', 'y','X']
pred_df['ds'] = data['ds']

pred_df['ds'] = pd.to_datetime(pred_df['ds']).dt.date
pred_df.columns = ['PRICE', 'DATE']
pred_df = pred_df[['DATE', 'PRICE']]
pred_df = pred_df.dropna()
print(pred_df)


            DATE      PRICE
0     2004-08-19   2.494294
1     2004-08-20   2.519454
2     2004-08-23   2.762083
3     2004-08-24   2.774289
4     2004-08-25   2.617850
...          ...        ...
4662  2023-02-27  90.107355
4663  2023-02-28  89.557274
4664  2023-03-01  90.177374
4665  2023-03-02  89.877324
4666  2023-03-03  92.757772

[4667 rows x 2 columns]


In [13]:
# Create an instance of Linear Regression and Fit the training datasets
from sklearn.linear_model import LinearRegression
lm = LinearRegression()

In [ ]:
#Import Required Libraries 
import streamlit as st
from snowflake.snowpark.session import Session
from plotly import graph_objs as go
import pandas as pd
import json
#import toml

#Extract credentials from secret file 
#secrets = toml.load("secrets.toml")
accountname = st.secrets["SNOWFLAKE"]["account"]
user = st.secrets["SNOWFLAKE"]["user"]
password = st.secrets["SNOWFLAKE"]["password"]
role = st.secrets["SNOWFLAKE"]["role"]
database = st.secrets["SNOWFLAKE"]["database"]
schema = st.secrets["SNOWFLAKE"]["schema"]
warehouse = st.secrets["SNOWFLAKE"]["warehouse"]

#Setting Menu options and web page configurations 
st.set_page_config(
     page_title="Stock Price Prediction",
     #layout="wide",
     page_icon="💹",
     initial_sidebar_state="expanded",
     menu_items={
         'Get Help': 'https://www.vinoddhole.com',
         'About': "The **Stock Price Prediction** App powered by AWS, Snowflake, Python, Snowpark and Streamlit"
     }
 )

# Create Session object
def create_session_object():
    connection_parameters = {
            "account": accountname,
            "user": user,
            "password": password,
            "role": role,
            "database": database,
            "schema": schema,
            "warehouse": warehouse,
            "ocsp_fail_open":"False"
        }

    session = Session.builder.configs(connection_parameters).create()

    print(session.sql('select current_warehouse(), current_database(), current_schema()').collect())
    return session

#Extract the data from SnowFlake load on pandas DataFrame 
def fetch_data(session):
    df = session.table('historical_prices').to_pandas()
    df.drop_duplicates(subset='DATE', keep="last",inplace=True)
    df = df.sort_values(by='DATE',ascending=False)
    return df
    
#Setting web page Title
st.title('Stock Price Prediction')

#Drop down selection for Ticker 
#Currently there is Only one option Google, can add more options like stocks = ('GOOG','AAPL')
stocks = ('GOOG',)
selected_stock = st.selectbox('Select Ticker', stocks)


if __name__ == "__main__":

    #Function call to connect to SnowFlake
    session = create_session_object()

    #Setting status progression text in webpage 
    data_load_state = st.text('Loading data...')
    
    #Function call to Get data 
    price_df = fetch_data(session)
    
    #Setting process completion text in webpage 
    data_load_state.text('Loading data... done!')

    st.subheader('Historical Prices')
    
    #Displaying Historical pricing data 
    st.dataframe(price_df)
       
    st.subheader('Historical Price Trend')

    #Creating Visualization for Historical Prices (This is exactly same code from Article 2)
    trace = go.Scatter(x=price_df['DATE'], y=price_df['CLOSE'],line_color='deepskyblue', name = 'Actual Prices')

    data = [trace]
    layout = dict(
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                        label='1m',
                        step='month',
                        stepmode='backward'),
                    dict(count=3,
                        label='3m',
                        step='month',
                        stepmode='backward'),
                    dict(count=6,
                        label='6m',
                        step='month',
                        stepmode='backward'),
                    dict(count=12,
                        label='1Y',
                        step='month',
                        stepmode='backward'),
                    dict(count=5,
                        label='5Y',
                        step='year',
                        stepmode='backward'),
                    dict(step="all")
                ])
            ),
            title='Date',
            rangeslider=dict(
                visible = True
            ), type='date'
        ),
        yaxis=dict(title='Closing Price')
    )
    fig = dict(data=data, layout=layout)
    st.plotly_chart(fig)

    #Slider bar for users to select no. future days Range 1 - 1825 days, default is 180 days 
    days = st.slider('Select No. of days for prediction', 1, 1852,180)
    
    #Yes/No Selection option to show Historical Prices
    show_history = st.selectbox(
        "Show Historical Predicted Prices?",
        ("N", "Y")
    )

    #On Button click run code inside if statement 
    if st.button('Forecast Predictions'):
        
        prediction_state = st.text('Predicting Future Prices...')

        #Calling snowflake stored procedure "sproc_predict_using_prophet(table,show_hist, days", which will predict future prices (Same code grabbed from Article 3)
        pred_list = session.sql(
                "call sproc_predict_using_regression('{}', '{}',{})".format('historical_prices',show_history, days)   
                ).collect()

        #Load the Prediction data from stored procedure into DataFrame 
        pred_df = pd.DataFrame(json.loads(pred_list[0][0]))
        
        
        pred_df[['DATE', 'Forecast']] = pred_df['forecast'].apply(pd.Series)
        st.subheader('Predicted Prices')
        pred_df=pred_df.drop['forecast']
        #Display the prediction output 
        st.dataframe(pred_df)
        st.subheader('Predicted Price Trend')
        
        
        
        
        
        
        pred_list = session.sql(
                "call sproc_predict_using_regression2('{}', '{}',{})".format('historical_prices',show_history, days)   
                ).collect()

        #Load the Prediction data from stored procedure into DataFrame 
        pred_df = pd.DataFrame(json.loads(pred_list[0][0]))
        data_sdf = session.table('historical_prices')
        data = data_sdf.select('DATE', 'CLOSE','OPEN').to_pandas()
        data.drop_duplicates(subset='DATE', keep="last", inplace=True)
        data.dropna(subset=['DATE'])
        data.sort_values(by='DATE', inplace=True)
        data.columns = ['ds', 'y','X']
        pred_df['ds'] = data['ds']
        pred_df['ds'] = pd.to_datetime(pred_df['ds']).dt.date
        pred_df.columns = ['CLOSE', 'DATE']
        pred_df = pred_df[['DATE', 'CLOSE']]
        pred_df = pred_df.dropna()
        
        st.subheader('Predicted Price Trend')


        trace0 = go.Scatter(x=price_df.dropna(subset=['DATE'])['DATE'], y=price_df.dropna(subset=['DATE'])['CLOSE'],line_color='deepskyblue', name='Actual Prices')

        trace1 = go.Scatter(x=pred_df['DATE'], y=pred_df['CLOSE'],line_color='lime', name='Predicted Prices')

        
        
        
        
        
        
        
        
        
        
        

       
        #Visualization of Actual Prices vs Predicted Prices (This is exactly same code from Article 2)
        data = [trace0, trace1]
        layout = dict(
            title='Actual Prices vs Predicted Prices',
            xaxis=dict(
                rangeselector=dict(
                    buttons=list([
                        dict(count=1,
                            label='1m',
                            step='month',
                            stepmode='backward'),
                        dict(count=3,
                            label='3m',
                            step='month',
                            stepmode='backward'),
                        dict(count=6,
                            label='6m',
                            step='month',
                            stepmode='backward'),
                        dict(count=12,
                            label='1Y',
                            step='month',                     
                            stepmode='backward'),
                        dict(count=5,
                            label='5Y',
                            step='year',
                            stepmode='backward'),
                        dict(step="all")
                    ])
                ),
                title='Date',
                rangeslider=dict(
                    visible = True
                ), type='date'
            ),
            yaxis=dict(title='Closing Price')
        )
        fig = dict(data=data, layout=layout)
        st.plotly_chart(fig)
        prediction_state.text('Prediction Done!')

In [49]:
def sproc_predict_using_regression2(session: Session,
                                   training_table: str,
                                   include_history: str,
                                   period: int) -> T.Variant:
    # Loading data into pandas dataframe
    data_sdf = session.table(training_table)
    data = data_sdf.select('DATE', 'CLOSE','OPEN').to_pandas()
    data.drop_duplicates(subset='DATE', keep="last", inplace=True)
    data.dropna(subset=['DATE'])

    data.sort_values(by='DATE', inplace=True)
    data.columns = ['ds', 'y','X']
    from sklearn.linear_model import LinearRegression
    # Actual model training
    
    model = LinearRegression()
    
    X = data['X'].values.reshape(-1, 1)  # Use this if you created 'ds_timestamp'
    
    y = data['y']
    
    model.fit(X, y)
    
    
    
    
    if include_history == 'Y':
        flag = True
    else:
        flag = False
 
    
    forecast = model.predict(X)
    
    result = { 'forecast': forecast.tolist()}
    
    return result
    


session.add_packages('snowflake-snowpark-python','scikit-learn')
# Registering the function as a Stored Procedure
sproc_predict_using_regression = session.sproc.register(func=sproc_predict_using_regression2,
                                                       name='sproc_predict_using_regression2',
                                                       is_permanent=True,
                                                       replace=True,
                                                       stage_location='@ML_MODELS',
                                                       packages=['snowflake-snowpark-python','scikit-learn'])
 
training_table = 'historical_prices'
show_history = 'N'
future_days = 4
 
pred_result = session.sql(
    "call sproc_predict_using_regression2('{}', '{}', {})".format(training_table, show_history, future_days)
).collect()
'''
# Accessing the prediction results
future_dates = pred_result[0][0]
forecast_values = pred_result[0][0]
 
#pred_df = pd.DataFrame({'DATE': future_dates, 'PRICE': forecast_values})
#pred_df['DATE'] = pred_df['DATE'].dt.date
print(future_dates,forecast_values)
# Access pred_df for the predictions
'''
pred_df = pd.DataFrame(json.loads(pred_result[0][0]))
data_sdf = session.table(training_table)
data = data_sdf.select('DATE', 'CLOSE','OPEN').to_pandas()
data.drop_duplicates(subset='DATE', keep="last", inplace=True)
data.dropna(subset=['DATE'])

data.sort_values(by='DATE', inplace=True)
data.columns = ['ds', 'y','X']
pred_df['ds'] = data['ds']

pred_df['ds'] = pd.to_datetime(pred_df['ds']).dt.date
pred_df.columns = ['PRICE', 'DATE']
pred_df = pred_df[['DATE', 'PRICE']]
pred_df = pred_df.dropna()
print(pred_df)


            DATE      PRICE
0     2004-08-19   2.494294
1     2004-08-20   2.519454
2     2004-08-23   2.762083
3     2004-08-24   2.774289
4     2004-08-25   2.617850
...          ...        ...
4662  2023-02-27  90.107355
4663  2023-02-28  89.557274
4664  2023-03-01  90.177374
4665  2023-03-02  89.877324
4666  2023-03-03  92.757772

[4667 rows x 2 columns]


In [48]:
from datetime import timedelta

from datetime import timedelta

def sproc_predict_using_regression(session: Session,
                                   training_table: str,
                                   include_history: str,
                                   period: int) -> T.Variant:
    # Loading data into pandas dataframe
    data_sdf = session.table(training_table)
    data = data_sdf.select('DATE', 'CLOSE', 'OPEN').to_pandas()
    data.drop_duplicates(subset='DATE', keep="last", inplace=True)
    data.dropna(subset=['DATE'], inplace=True)  # Ensure 'DATE' column has no NaN values

    data.sort_values(by='DATE', inplace=True)
    data.columns = ['ds', 'y', 'X']

    from sklearn.linear_model import LinearRegression

    # Actual model training
    model = LinearRegression()
    
    # Assuming 'X' represents the feature data, and 'y' represents the target variable
    X = data['X'].values.reshape(-1, 1)
    y = data['y']
    model.fit(X, y)
    
    # Future prediction for 'period' days
    future_dates = pd.date_range(start=data['ds'].max() + timedelta(days=1), periods=period)
    future_X = np.arange(len(data), len(data) + period).reshape(-1, 1)
    future_forecast = model.predict(future_X)
    
    # Creating a DataFrame for the forecasted dates and values
    forecast_dates = [date.strftime('%Y-%m-%d') for date in future_dates]
    forecast_data = {'DATE': forecast_dates, 'Forecast': future_forecast}
    forecast_df = pd.DataFrame(forecast_data)
    
    result = {'forecast': forecast_df.to_dict(orient='records')}
    
    return result

session.add_packages('snowflake-snowpark-python','scikit-learn')
# Registering the function as a Stored Procedure
sproc_predict_using_regression = session.sproc.register(func=sproc_predict_using_regression,
                                                       name='sproc_predict_using_regression',
                                                       is_permanent=True,
                                                       replace=True,
                                                       stage_location='@ML_MODELS',
                                                       packages=['snowflake-snowpark-python','scikit-learn'])
 
training_table = 'historical_prices'
show_history = 'N'
future_days = 4
 
pred_result = session.sql(
    "call sproc_predict_using_regression('{}', '{}', {})".format(training_table, show_history, future_days)
).collect()
'''
# Accessing the prediction results
future_dates = pred_result[0][0]
forecast_values = pred_result[0][0]
 
#pred_df = pd.DataFrame({'DATE': future_dates, 'PRICE': forecast_values})
#pred_df['DATE'] = pred_df['DATE'].dt.date
print(future_dates,forecast_values)
# Access pred_df for the predictions
'''
pred_df = pd.DataFrame(json.loads(pred_result[0][0]))

print(pred_df)


                                            forecast
0  {'DATE': '2023-12-12', 'Forecast': 4671.735316...
1  {'DATE': '2023-12-13', 'Forecast': 4672.735473...
2  {'DATE': '2023-12-14', 'Forecast': 4673.735630...
3  {'DATE': '2023-12-15', 'Forecast': 4674.735787...
